In [1]:
from omegaconf import OmegaConf
import hydra
import torch


In [2]:
def load_model():
    V = 2
    RUN_ID = "nw6nhpa2"
    EPOCH = "0199"
    ckpt = torch.load(f"../../artifacts/model-{RUN_ID}:v0/epoch_{EPOCH}.ckpt")["model"]
    ckpt["pose_head.linear.weight"] = ckpt.pop("decoder_pred.weight")
    ckpt["_patch_loss.sigma"] = torch.tensor([0.1, 0.1, 0.1, 0.1])
    # ckpt["segment_embed"] = ckpt["segment_embed"][:V]
    ckpt["segment_embed"] = torch.zeros_like(ckpt["segment_embed"])[:V]

    cfg = OmegaConf.load(f"../../artifacts/model-{RUN_ID}:v0/config.yaml")
    cfg["model"]["segment_embed_mode"] = (
        "permute" if cfg["model"].pop("permute_segment_embed") else "fixed"
    )
    model = hydra.utils.instantiate(
        cfg["model"],
        _target_="src.models.components.partmae_v6.PARTMaskedAutoEncoderViT",
        verbose=True,
        num_views=V,
        # lambda_dino=0.1
    )
    model.load_state_dict(ckpt, strict=False)
    return model

In [3]:
model: torch.nn.Module = load_model()

In [4]:
model.dino_head

In [6]:
from src.models.components.utils.partmae_v6_simdino import load_partmaev6_from_simdino

ckpt_path = "../../artifacts/vitb16_SimDINOv1_gpu8_bs64_ep100.pth"
# _ = load_partmaev6_from_simdino(model, ckpt_path)


In [7]:
new_state = {
    "global_step": 5,
}
torch.save(new_state, "new_state.pth")

In [8]:
from lightning.fabric import Fabric
fabric = Fabric()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
state = {
    "model": torch.ones(3),
    "global_step": 2,
}
print(state["global_step"])
state_dict = fabric.load("new_state.pth", state, strict=True)
print(state["global_step"])
print(state_dict.keys())

2


KeyError: "The requested state contains a key 'model' that does not exist in the loaded checkpoint. To disable strict loading, set `strict=False`."

In [ ]:
RUN_ID = "nw6nhpa2"
cfg = OmegaConf.load(f"../../artifacts/model-{RUN_ID}:v0/config.yaml")
cfg["model"]["segment_embed_mode"] = (
    "permute" if cfg["model"].pop("permute_segment_embed") else "fixed"
)
model = hydra.utils.instantiate(
    cfg["model"],
    _target_="src.models.components.utils.partmae_v6_simdino.PARTMaskedAutoEncoderViTFromSimDINO",
    verbose=True,
    num_views=2,
    lambda_dino=0.1
)
state = {"model": model}
ckpt_path = "../../artifacts/vitb16_SimDINOv1_gpu8_bs64_ep100.pth"
state_dict = torch.load(ckpt_path)
fabric = Fabric()
print(model.dino_head.mlp[4].bias.sum())
print(state_dict.keys())
fabric.load_raw(ckpt_path, model)
print(model.dino_head.mlp[4].bias.sum())

tensor(0., grad_fn=<SumBackward0>)
dict_keys(['teacher'])
tensor(0.1227, grad_fn=<SumBackward0>)


In [22]:
state["global_step"]
state["model"]

tensor([1., 1., 1.])

In [ ]:
# ckpt = torch.load("../../artifacts/vitb16_SimDINOv1_gpu8_bs64_ep100.pth")
# state_dict = ckpt["teacher"]
# 
# # 1. Remove the prefix "_orig_mod." from the keys
# unwanted_prefix = "_orig_mod."
# for k, v in list(state_dict.items()):
#     if k.startswith(unwanted_prefix):
#         state_dict[k[len(unwanted_prefix) :]] = state_dict.pop(k)
# 
# # state_dict.keys()
# # 2. replace prefix head with dino_head
# for k, v in list(state_dict.items()):
#     if k.startswith("head."):
#         state_dict[k.replace("head.", "dino_head.")] = state_dict.pop(k)
# 
# # 3. replace remove prefix  backbone
# for k, v in list(state_dict.items()):
#     if k.startswith("backbone."):
#         state_dict[k.replace("backbone.", "")] = state_dict.pop(k)
# 
# # 4. reshape patch_embed.proj.weight
# state_dict["patch_embed.proj.weight"] = state_dict["patch_embed.proj.weight"].reshape(
#     state_dict["patch_embed.proj.weight"].shape[0], -1
# )
# 